In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import nltk

import matplotlib.pyplot as plt
%matplotlib inline

from importlib import reload

from ReviewFeatureExtractor import ExtractBoW

### Hyperparameters

In [2]:
VOC_SIZE = 100000

### All Words

In [3]:
e = ExtractBoW(voc_size=VOC_SIZE)

e.load_word_vecs('data/wiki-news-300d-1M.vec')

### Need to iterate through each business and person

In [10]:
df_reviews = pd.read_pickle('results/tor_reviews.p')  # reviews
df_bus = pd.read_pickle('results/rest_univ.p')  # businesses